In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import torch

In [ ]:
df = pd.read_csv('../public_data/train/track_a/eng.csv')

In [ ]:
import string

def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

df['text'] = df['text'].apply(remove_punctuation)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def lol(sentence):
    encoded = tokenizer(text, 
                       padding='max_length',
                       max_length=512,
                       truncation=True,
                       return_tensors='pt')
    return encoded.input_ids[0]

def lol2(sentence):
    encoded = tokenizer(text, 
                       padding='max_length',
                       max_length=512,
                       truncation=True,
                       return_tensors='pt')
    return encoded.attention_mask[0]
# lol(text)
df['input_ids'] = df['text'].apply(lol)
df['attention_mask'] = df['text'].apply(lol2)

In [ ]:
class TweetsDataset(torch.utils.data.Dataset):
    def __init__(self, df, target_columns, feature_columns):
        self.df = df.copy()
        self.target_columns = target_columns
        self.feature_columns = feature_columns
        
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return row[self.feature_columns].values.tolist(), torch.tensor(row[self.target_columns].values.tolist(), dtype=torch.float32)

In [237]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

emotions_list = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
input_list = ['input_ids', 'attention_mask']
dataset = TweetsDataset(df, emotions_list, input_list)

train_size = int(.7 * len(dataset))
validation_size = len(dataset) - train_size

# Create splits
train_dataset, validation_dataset = random_split(
    dataset, 
    [train_size, validation_size],
    generator=torch.Generator().manual_seed(42)
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=64,
)

In [ ]:
from copy import deepcopy

# Create normalized coccurance matrix (3.1)
cooccurance_matrix = [[0] * 5 for i in range(5)]

for _, row in df.iterrows():
    for index, emotion in enumerate(emotions_list):
        if row[emotion] == 1:
            for index2, emotion2 in enumerate(emotions_list):
                cooccurance_matrix[index][index2] += row[emotion2]
                
normalized_m = deepcopy(cooccurance_matrix)
for i in range(len(normalized_m)):
    k = sum(df[emotions_list[i]])
    for j in range(len(normalized_m)):
        normalized_m[i][j] /= k

mu = 0.2
binarized_m = deepcopy(normalized_m)
for i in range(5):
    for j in range(5):
        if binarized_m[i][j] > mu:
            binarized_m[i][j] = 1
        else:
            binarized_m[i][j] = 0

w = .35
mitigate_oversmooth_m = deepcopy(binarized_m)
for i in range(5):
    row_sum = sum([i for i in mitigate_oversmooth_m[i]])
    for j in range(5):
        if i != j:
            mitigate_oversmooth_m[i][j] /= row_sum
        else:
            mitigate_oversmooth_m[i][j] -= w

adj = torch.tensor(mitigate_oversmooth_m)

# Calculate degree matrix
degrees = adj.sum(dim=1)

# Calculate D^(-1/2)
# Add small epsilon to prevent division by zero
d_inv_sqrt = torch.pow(degrees + 1e-7, -0.5)

# Convert to diagonal matrix
d_inv_sqrt = torch.diag(d_inv_sqrt)

# Normalized adjacency: D^(-1/2) A D^(-1/2)
q = d_inv_sqrt @ adj @ d_inv_sqrt

In [ ]:
import numpy as np

def load_glove_vectors(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_dict[word] = vector
    return embeddings_dict

def encode_words(words, embeddings_dict, embedding_dim=300):
    # Initialize zero vector for unknown words
    unknown_vector = np.zeros(embedding_dim)
    
    # Encode each word
    encoded = []
    for word in words:
        word = word.lower()  # GloVe vocab is lowercase
        vector = embeddings_dict.get(word, unknown_vector)
        encoded.append(vector)
    
    return np.array(encoded)

glove_file = "../glove.6B.300d.txt"
embeddings = load_glove_vectors(glove_file)

vectors = encode_words(emotions_list, embeddings)

In [ ]:
import math
import torch.nn as nn
from transformers import BertModel

class EmoGraph(torch.nn.Module):
    def __init__(self, ee):
        super().__init__()
        self.w1 = torch.nn.Parameter(torch.randn(300, 768))
        nn.init.xavier_uniform_(self.w1)
        self.pp = [self.w1]
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.ee = ee
    def forward(self, input_ids, attention_mask, matrix):
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        bert_embeddings = bert_output.last_hidden_state[:, 0, :]

        h = nn.functional.relu(matrix @ self.ee @ self.w1)

        logits = bert_embeddings @ h.t()
        out = nn.functional.sigmoid(logits)
        return out

In [ ]:
vectors.shape

In [ ]:
model = EmoGraph(vectors)
loss_fn = torch.nn.BCELoss()

bert_optimizer = torch.optim.Adam(
    model.bert.parameters(), 
    lr=2e-5
)
gcn_optimizer = torch.optim.AdamW(
    model.pp,
    lr=0.001,
)

In [238]:
losses = []
i_s = []
for i in range(50):
    model.train()
    for idx, data in enumerate(train_loader):
        inputs, labels = data
        input_id, attention = inputs
        optimizer.zero_grad()

        output = model(input_id, attention, q)
        # print(output.shape)
        # break
        loss = loss_fn(output, labels)
        loss.backward()

        optimizer.step()
        print(loss)
    losses.append(loss.item())
    i_s.append(i)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(i_s, losses)

In [ ]:
thresh = .4
test_df = pd.read_csv('../public_data/dev/track_a/eng_a.csv')
all_results = []

for idx, row in test_df.iterrows():
    e = embedding_model.encode(row['text'])
    e = np.append(e, [sentiment(row['text'])])
    pred = model(torch.tensor(e)).tolist()
    res = [1 if k > thresh else 0 for k in pred]
    lol = [row['id']]
    lol.extend(res)
    all_results.append(lol)
    # print(all_results)
    # break
# for i in 
output_df = pd.DataFrame(all_results, columns=['id','Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])

output_df.to_csv('../outputs/pred_eng_a.csv', index=False)